In [3]:
!nvidia-smi

Fri Apr  9 13:17:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install regex requests

In [5]:
import functools
import importlib
from copy import deepcopy

In [ ]:
 pip install hydra-core --upgrade

In [7]:
import functools
import importlib


dependencies = [
    "dataclasses",
    "hydra",
    "numpy",
    "omegaconf",
    "regex",
    "requests",
    "torch",
]


missing_deps = []
for dep in dependencies:
    try:
        importlib.import_module(dep)
    except ImportError:
        if dep == "hydra":
            dep = "hydra-core"
        missing_deps.append(dep)

In [8]:
import torch

In [ ]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()

In [ ]:
pip install datasets

In [ ]:
pip install allennlp

In [12]:
from datasets import load_dataset, load_metric
from allennlp.common.util import lazy_groups_of
import numpy as np
from fairseq.data.data_utils import collate_tokens
import heapq
from operator import itemgetter

In [ ]:
actual_task = "mnli"
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer1 = AutoTokenizer.from_pretrained("roberta-large-mnli")

In [16]:
dataset_label_filter = 0

In [17]:
subset_dev_dataset = []
for instance in dataset['train']:
    if instance['label'] == dataset_label_filter:
        if np.random.rand() < 0.05:
          d = {'hypothesis':instance['hypothesis'], 'label':2-instance['label'], 'premise':instance['premise']}
          subset_dev_dataset.append(d)

In [18]:
target_label = 0

In [19]:
len(subset_dev_dataset)

6572

In [ ]:
subset_dev_dataset

In [21]:
from torch.utils.data import Dataset, DataLoader

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
roberta.to(device)

In [43]:
def evaluate_batch(model, batch, trigger_token=None):
    res_batch = []
    for i in range(len(batch[0]['premise']) // 8):
      d = {'hypothesis':batch[0]['hypothesis'][8*i:min(8*(i+1), len(batch[0]['hypothesis']))],
           'label':batch[0]['label'][8*i:min(8*(i+1), len(batch[0]['label']))],
           'premise':batch[0]['premise'][8*i:min(8*(i+1), len(batch[0]['premise']))]}
      res_batch.append(d)
    length = len(batch[0]['premise'])
    del batch


    right = 0.
    for j in range(len(res_batch)):
      sentences = []
      for i in range(8):
        s = ''
        if trigger_token != None:
          for k in trigger_token:
            s += k
          s += ' '
        sentences.append([s + res_batch[j]['premise'][i], res_batch[j]['hypothesis'][i]])
      sentences = collate_tokens([roberta.encode(sent1, sent2) for sent1, sent2 in sentences], pad_idx=1)
      tmp = roberta.predict('mnli', sentences).detach().cpu()
      pred = tmp.argmax(axis=1)
      for i in range(len(pred)):
          right += (pred[i] == res_batch[j]['label'][i])
    right /= length
    return right.detach().cpu().numpy()

In [44]:
for batch in lazy_groups_of(DataLoader(subset_dev_dataset, batch_size=len(subset_dev_dataset)), group_size=1):
  print("Nothing", ": ", evaluate_batch(roberta, batch))

Nothing :  0.97945833


In [39]:
trigger_tokens = [' nobody', ' never', ' sad', ' scared', ' championship']
for i in trigger_tokens:
  for batch in lazy_groups_of(DataLoader(subset_dev_dataset, batch_size=len(subset_dev_dataset)), group_size=1):
    print(i, ": ", evaluate_batch(roberta, batch, trigger_token=[i]))

 nobody :  0.97671944
 never :  0.9770237
 sad :  0.9771759
 scared :  0.97671944
 championship :  0.9771759


In [49]:
for i in trigger_tokens:
  for j in trigger_tokens:
    for batch in lazy_groups_of(DataLoader(subset_dev_dataset, batch_size=len(subset_dev_dataset)), group_size=1):
      print(i, ", ", j, ": ", evaluate_batch(roberta, batch, trigger_token=[i, j]))

 joyously ,   joyously :  0.97732806
 joyously ,   anticipating :  0.97458917
 joyously ,   talented :  0.9786975
 joyously ,   impress :  0.97808886
 joyously ,   inspiring :  0.9776324
 anticipating ,   joyously :  0.9751978
 anticipating ,   anticipating :  0.97398055
 anticipating ,   talented :  0.9762629
 anticipating ,   impress :  0.97656727
 anticipating ,   inspiring :  0.9759586
 talented ,   joyously :  0.97458917
 talented ,   anticipating :  0.97413266
 talented ,   talented :  0.9764151
 talented ,   impress :  0.97656727
 talented ,   inspiring :  0.9758065
 impress ,   joyously :  0.974437
 impress ,   anticipating :  0.97458917
 impress ,   talented :  0.97611076
 impress ,   impress :  0.97656727
 impress ,   inspiring :  0.9759586
 inspiring ,   joyously :  0.97458917
 inspiring ,   anticipating :  0.9742848
 inspiring ,   talented :  0.97656727
 inspiring ,   impress :  0.97671944
 inspiring ,   inspiring :  0.9764151


In [50]:
for i in range(15):
  tt = []
  for j in np.random.choice(5, 4):
    tt.append(trigger_tokens[j])
  for batch in lazy_groups_of(DataLoader(subset_dev_dataset, batch_size=len(subset_dev_dataset)), group_size=1):
    s = tt[0]
    for j in tt[1:]:
      s += ', ' + j
    print(s, ": ", evaluate_batch(roberta, batch, trigger_token=tt))

 anticipating,  impress,  inspiring,  joyously :  0.9762629
 impress,  joyously,  anticipating,  inspiring :  0.9738284
 impress,  joyously,  joyously,  anticipating :  0.9703287
 talented,  impress,  anticipating,  joyously :  0.97534996
 impress,  joyously,  talented,  joyously :  0.9776324
 talented,  anticipating,  anticipating,  inspiring :  0.9764151
 anticipating,  talented,  anticipating,  impress :  0.97611076
 inspiring,  joyously,  inspiring,  anticipating :  0.9748935
 talented,  anticipating,  impress,  talented :  0.97534996
 joyously,  joyously,  anticipating,  joyously :  0.9770237
 impress,  talented,  talented,  joyously :  0.97854537
 impress,  inspiring,  talented,  joyously :  0.9764151
 impress,  joyously,  anticipating,  impress :  0.97458917
 anticipating,  joyously,  joyously,  anticipating :  0.9704808
 inspiring,  joyously,  inspiring,  joyously :  0.97732806
